In [2]:
from scipy.special import roots_jacobi, eval_jacobi
import numpy as np
import scipy
import math

### Define functions for D operator:

In [3]:
def int_points(x_range, npts, dx, M):
    
    k = 0
    p_order = npts - 1
    xinterior, w = roots_jacobi(p_order - 1,1,1) # returns interior GLL nodes from range -1 and 1
    GLL_points = np.pad(xinterior, (1, 1), 'constant', constant_values=(-1, 1))
    GLL_dist = np.array([np.abs(GLL_points[0] - value) for value in GLL_points])
    
    return_array = np.zeros([M, npts])
    
    for i in x_range[:-1]:
        array = np.array([i])
        for j in GLL_dist[1:]: 
            scaled_point = ((j / 2) * (dx)) + i # Note: 2 here is the distance from -1 to 1
            array = np.concatenate((array, np.array([scaled_point])))
            
        return_array[k, :] = array
        k = k + 1
    
    return(return_array)

In [4]:
def vandermonde(npts_values, return_type):
    # For the monomial basis: 
    if return_type == 'Monomial':
        return(np.vander(npts_values, increasing = True))
    elif return_type == 'Legendre':
        return(np.polynomial.legendre.legvander(npts_values, len(npts_values) - 1))

In [5]:
def Ld_vandermonde(npts_values): 
    
    LP = np.polynomial.legendre
    v_matrix = np.polynomial.legendre.legvander(npts_values, len(npts_values)-1)
    d_v_matrix = np.zeros_like(v_matrix)

    for i in range(len(npts_values)):
        # Coefficients for the i-th Legendre polynomial (e.g., [0, 0, 1] for P_2)
        coeffs = np.zeros(len(npts_values))
        coeffs[i] = 1

        # Compute the derivative of the i-th Legendre polynomial
        deriv_coeffs = LP.legder(coeffs, m=1) # m=1 for first derivative

        # Evaluate the derivative at the points x
        d_v_matrix[:, i] = LP.legval(npts_values, deriv_coeffs)
        
    return(d_v_matrix)

In [6]:
def return_D(V1, V2):
    return(np.matmul(V2, np.linalg.inv(V1)))

In [7]:
# Initialize the elements and the Guassian function:

L = 12 # Denotes total length in [0, L]
M = 4 # Denotes number of elements 
x_range = np.arange(0, L + (L/M), (L/M)) # Define the element endpoints

npts = 4
p_order = npts - 1
xinterior, w = roots_jacobi(p_order - 1,1,1) # returns interior GLL nodes from range -1 and 1
GLL_points = np.pad(xinterior, (1, 1), 'constant', constant_values=(-1, 1))

# Note: The D matrix is only created once as it operates on the reference element
V1 = vandermonde(GLL_points, 'Legendre')
V2 = Ld_vandermonde(GLL_points)
D = return_D(V1, V2)

# The following is used in creating the init condition: 
npts = int_points(x_range, 4, L/M, M)

In [8]:
def quad_weights(npts_values):
    
    # Values in f_array result from exact integration:
    f_array = np.zeros(len(npts_values))
    f_array[0] = 2
    
    # Define basis matrix for Legendre polynomials: 
    V = np.polynomial.legendre.legvander(npts_values, len(npts_values) - 1).transpose()
    
    return(np.matmul(np.linalg.inv(V), f_array.transpose()))

### DG:

In [71]:
C = (L / (2 * M))

-D[0, 0] + (1 / (2 * quad_weights(GLL_points)[-1] * C)) - (1 / (quad_weights(GLL_points)[-1] * C))

1.0

In [36]:
# Note: Have to come back to confirm the +/- signs; check indexing

def global_DG(D, M, p, L, quad_weights):
    
    array = np.zeros((M * (p + 1), M * (p + 1)))
    C = (L / (2 * M))
    row_i = 0
    col_i = 0
    
    for i in np.arange(M):
        if i == 0:
            array[row_i : row_i + (p+1), col_i : col_i + (p+1)] = -D
            
            # Right boundary correction:
            array[row_i + p, row_i + p] = -D[-1, -1] - (1 / (2 * quad_weights[-1] * C)) + (1 / (quad_weights[-1] * C))
            array[row_i + p, row_i + p + 1] = -(1 / (2 * quad_weights[-1] * C))
            
            # Left boundary correction:
            array[row_i, row_i] = -D[0, 0] + (1 / (2 * quad_weights[0] * C)) - (1 / (quad_weights[0] * C))
            
            # Periodic boundaries:
            array[0, -1] = (1 / (2 * quad_weights[0] * C))
            
        elif i == (M-1):
            array[row_i : row_i + (p+1), col_i : col_i + (p+1)] = -D
            
            # Left boundary correction:
            array[row_i, row_i] = -D[0, 0] + (1 / (2 * quad_weights[0] * C)) - (1 / (quad_weights[0] * C))
            array[row_i, row_i - 1] = (1 / (2 * quad_weights[0] * C))
            
            # Right boundary correction:
            array[row_i + p, row_i + p] = -D[-1, -1] - (1 / (2 * quad_weights[-1] * C)) + (1 / (quad_weights[-1] * C))
            
            # Periodic boundaries:
            array[-1, 0] = -(1 / (2 * quad_weights[-1] * C))
            
        else:
            array[row_i : row_i + (p+1), col_i : col_i + (p+1)] = -D
            
            # Right boundary correction:
            array[row_i + p, row_i + p] = -D[-1, -1] - (1 / (2 * quad_weights[-1] * C)) + (1 / (quad_weights[-1] * C))
            array[row_i + p, row_i + p + 1] = -(1 / (2 * quad_weights[-1] * C))
        
            # Left boundary correction:
            array[row_i, row_i] = -D[0, 0] + (1 / (2 * quad_weights[0] * C)) - (1 / (quad_weights[0] * C))
            array[row_i, row_i - 1] = (1 / (2 * quad_weights[0] * C))
        
        row_i = row_i + (p+1)
        col_i = col_i + (p+1)
        
    return(array)


In [72]:
np.linalg.eigvals(global_DG(D, M, p_order, L, quad_weights(GLL_points)))

array([-2.22044605e-16+3.68897423e+00j, -2.22044605e-16-3.68897423e+00j,
        5.55111512e-17+3.37029657e+00j,  5.55111512e-17-3.37029657e+00j,
       -4.44089210e-16+3.16227766e+00j, -4.44089210e-16-3.16227766e+00j,
        3.62838473e-01+2.23949269e+00j,  3.62838473e-01-2.23949269e+00j,
       -3.62838473e-01+2.23949269e+00j, -3.62838473e-01-2.23949269e+00j,
        0.00000000e+00+1.62514647e+00j,  0.00000000e+00-1.62514647e+00j,
       -3.10233715e-17+1.95077333e-08j, -3.10233715e-17-1.95077333e-08j,
       -6.93889390e-17+7.90011146e-01j, -6.93889390e-17-7.90011146e-01j])

In [45]:
D.transpose()

array([[-3.00000000e+00, -8.09016994e-01,  3.09016994e-01,
        -5.00000000e-01],
       [ 4.04508497e+00, -1.11022302e-16, -1.11803399e+00,
         1.54508497e+00],
       [-1.54508497e+00,  1.11803399e+00,  1.11022302e-16,
        -4.04508497e+00],
       [ 5.00000000e-01, -3.09016994e-01,  8.09016994e-01,
         3.00000000e+00]])

### CG:

In [79]:
def global_CG(D, M, p):
    
    row_i = 0
    col_i = 0
    edg_i = p
    array = np.zeros(((M * p), (M * p)))
    
    for i in np.arange(M):
        if (i % 2) == 0:
            array[row_i : row_i + (p+1), 
                  col_i : col_i + (p+1)] = -D
            
            val_1 = -D[-1, -1]
            
        else: 
            if i < M-1:
                array[row_i : row_i + (p+1), 
                      col_i : col_i + (p+1)] = -D
            
                val_2 = -D[0, 0]
            
                array[edg_i, edg_i] = val_1 + val_2
                array[edg_i, :] = (array[edg_i, :] / 2) # For centered average of u', divide all terms by 2
                edg_i = edg_i + p
                
                
    

In [ ]:
    row_i = row_i + (p)
        col_i = col_i + (p)
        
        print(i)
        
    # Implement periodic boundary conditions:
    
    # Left boundary:
    array[0, -3:] = -D[-1, 0:3]
    array[0, 0] = array[0, 0] + (-D[-1, -1])
    array[0, :] = array[0, :] / 2
    
    # Right boundary:
    array[-1, 0:3] = -D[0, 1:]
    array[-1, -1] = array[-1, -1] + (-D[0, 0])
    array[-1, :] = array[-1, :] / 2
    
    return(array)

In [ ]:
def global_CG(D, M, p):
    
    row_i = 0
    col_i = 0
    edg_i = p
    array = np.zeros(((M * p), (M * p)))
    
    for i in np.arange(M):
        if i == 0:
            array[row_i : row_i + (p+1), 
                  col_i : col_i + (p+1)] = -D
            
        elif i == M-1:
            array[row_i : row_i + (p+1), 
                  col_i : col_i + (p+1)] = -D[:3, :3]
            
            array[-4:, 0] = -D[-1 , :]
    
        else:
            array[row_i : row_i + (p+1), 
                  col_i : col_i + (p+1)] = -D
            
            array[edg_i, edg_i] = -D[0, 0] - D[-1, -1]
            array[edg_i, :] = (array[edg_i, :] / 2) # For centered average of u', divide all terms by 2
            edg_i = edg_i + p
            
        row_i = row_i + (p)
        col_i = col_i + (p)    
        